In [ ]:
## Plotly and Dash Loading
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import dash_leaflet as dl
import dash_leaflet.express as dlx
from dash_extensions.javascript import assign
from dash import Dash
from dash_extensions.javascript import arrow_function

import json

# Pandas and geopandas stuff
import pandas as pd
import geopandas as gpd
from geopy.geocoders import Nominatim
import geopandas as gpd
import fiona
from arcgis.gis import GIS
from geopy.distance import geodesic
import shapely.geometry
import geobuf

In [ ]:
housing = pd.read_csv('../Data/house_coordinates.csv')
housing = housing.head(10)
housing_geo = gpd.GeoDataFrame(
    housing, geometry=gpd.points_from_xy(housing.longitude, housing.latitude))

In [ ]:
housing = json.loads(housing_geo.to_json())

In [ ]:
lon_min = -93.7638;
lon_max = -93.4449;
lat_min = 41.9503;
lat_max = 42.1089;

def clean_Ames_location(df_place):
    '''
    DOCSTRING
    This cleans up the geopandas file for all the location data.
    It removes places that are more/less than a specific area based on lat and long.
    It then converts the files to JSON and then a GEOBUF, which will allow for faster loading.
    The boundary latitudes and longitudes are given by the global variables "lon, lat"_"min, max".
 
    INPUT:
    Geopandas data frame: Includes a geometry column
    
    OUTPUT:
    Geobuf file, with only locations within the specified area.
    '''
    df_place = df_place[(df_place['geometry'].x >= lon_min) &
                        (df_place['geometry'].x <= lon_max) & 
                        (df_place['geometry'].y >= lat_min) & 
                        (df_place['geometry'].y <= lat_max)]; 
    
### TO STOP CONVERSION TO GEOJSON, COMMENT OUT THIS LINE   
    df_place = json.loads(df_place.to_json())
    
### TO STOP CONVERSION TO GEOBUF, COMMENT OUT THIS LINE
#     df_place = dlx.geojson_to_geobuf(df_place)
    
    return df_place

In [ ]:
gis_osm_pofw = gpd.read_file(
    '../GeoJsonFiles/iowa-latest-free/gis_osm_pofw_free_1.shp')
churches = clean_Ames_location(gis_osm_pofw)

In [ ]:
app = JupyterDash(__name__)


geojson_filter = assign(
    "function(feature, context){{return ['903453080'].includes(feature.properties.PID);}}")



draw_church = assign("""function(feature, latlng){
const icon_church = L.icon(/assets/Church_symbol_gray.svg`, iconSize: [64, 48]});
return L.marker(latlng, {icon: cross});
}""")


app.layout = html.Div(children=[
    dl.Map([
    dl.TileLayer(),
    dl.LayersControl(
        [dl.Overlay(
            dl.LayerGroup(
                dl.GeoJSON(
                    data=housing, 
#                     options=dict(filter=geojson_filter), 
                    id="hous", cluster=True, zoomToBoundsOnClick=True)
            ), name = "Housing", checked = "True"
        ) #,
#          dl.Overlay(
#             dl.LayerGroup(
#                 dl.GeoJSON(
#                     data=churches, id="church", cluster=True, zoomToBoundsOnClick=True)
#             ), name = "Churchs", checked = "False"
#         )
        ]
     )
    ],
        center=[42.03, -93.64], zoom=12,
        style={'width': '100%', 'height': '50vh', 'margin': "auto", "display": "block"}
    ),
    
    # This just ensures we are getting the PID when clicked
    html.Div(id="PIDs"),
    
# This is adding a slider
    html.Label([
        "Price Range",
        dcc.RangeSlider(
            id='price',
            min=1000,
            max=500000,
            step=100,
            value=[100000, 200000]
        ),
    html.Div(id='output-container-range-slider')
    ])
])


            
            #             dl.GeoJSON(
#                 data=housing, options=dict(
#                 filter=geojson_filter),
#                        id="housing")),
#                     name="Housing", checked=True)]
#          ])

@app.callback(Output("PIDs", "children"), [Input("hous", "click_feature")])
def house_click(feature):
    if feature is not None:
        return f"You clicked {feature['properties']['PID']}"

    
    

app.run_server(mode='inline')

In [ ]:
dict(geojson_filter)

In [ ]:
# filter=geojson_filter
housing

In [ ]:
cities = [dict(name="Aalborg", lat=57.0268172, lon=9.837735),
          dict(name="Aarhus", lat=56.1780842, lon=10.1119354),
          dict(name="Copenhagen", lat=55.6712474, lon=12.5237848)]

dd_options = [dict(value=c["name"], label=c["name"]) for c in cities]
dd_defaults = [o["value"] for o in dd_options]
geojson = dlx.dicts_to_geojson([{**c, **dict(tooltip=c['name'])} for c in cities])

In [ ]:
dd_options

In [ ]:
geojson['features'][0]['properties']

In [ ]:
geojson_filter = assign("function(feature, context){return context.props.hideout.includes(feature.properties.name);}")
geojson_filter['variable']

In [ ]:
dict(filter=geojson_filter)

In [ ]:
housing['features'][0]['properties']


In [ ]:
def get_tuple_from_value(my_dict):
    new_dict = {}
    for term, nested_dict in my_dict.items():
        for id, n in nested_dict.items():
            new_dict[id] = [term, n]
    return new_dict

In [ ]:
housing['features']

In [ ]:
dd_options = [dict(value=c['properties']['PID'], label=c['properties']['PID']) for c in housing['features']]
dd_defaults = [o["value"] for o in dd_options]
dd_defaults


In [ ]:
dd_defaults

In [ ]:
app = JupyterDash(__name__)


# Create drop down options.
# dd_options = [dict(value=c["PID"], label=c["PID"]) for c in housing]
# dd_defaults = [o["value"] for o in dd_options]

geojson_filter = assign(
    "function(feature, context){{return context.props.hideout.includes(feature.properties.PID);}}")

app.layout = html.Div(children=[
    dl.Map([
    dl.TileLayer(),
    dl.LayersControl(
        [dl.Overlay(
            dl.LayerGroup(
                dl.GeoJSON(
                    data=housing, 
                    options=dict(filter=geojson_filter), 
                    id="housing_id", cluster=True, zoomToBoundsOnClick=True)
            ), name = "Housing", checked = "True"
        ) #,
#          dl.Overlay(
#             dl.LayerGroup(
#                 dl.GeoJSON(
#                     data=churches, id="church", cluster=True, zoomToBoundsOnClick=True)
#             ), name = "Churchs", checked = "False"
#         )
        ]
     )
    ],
        center=[42.03, -93.64], zoom=12,
        style={'width': '100%', 'height': '50vh', 'margin': "auto", "display": "block"}
    ),
    
    # This just ensures we are getting the PID when clicked
    html.Div(id="PIDs"),
    dcc.Dropdown(id="dd", value=dd_defaults, options=dd_options, clearable=False, multi=True),
    
# This is adding a slider
    html.Label([
        "Price Range",
        dcc.RangeSlider(
            id='price',
            min=1000,
            max=500000,
            step=100,
            value=[100000, 200000]
        ),
    html.Div(id='output-container-range-slider')
    ])
])


            
            #             dl.GeoJSON(
#                 data=housing, options=dict(
#                 filter=geojson_filter),
#                        id="housing")),
#                     name="Housing", checked=True)]
#          ])

@app.callback(Output("PIDs", "children"), [Input("hous", "click_feature")])
def house_click(feature):
    if feature is not None:
        return f"You clicked {feature['properties']['PID']}"

app.clientside_callback("function(x){return x;}", Output("housing_id", "hideout"), Input("dd", "value"))
    

app.run_server(mode='inline')

In [ ]:
app = JupyterDash(__name__)


geojson_filter = assign(
    "function(feature, context){{return ['903453080'].includes(feature.properties.PID);}}")



# draw_church = assign("""function(feature, latlng){
# const icon_church = L.icon(
# {iconUrl: `https://commons.wikimedia.org/wiki/Category:Religious_map_symbols#/media/File:Church_symbol_gray.svg`, 
# iconSize: [64, 48]}
# );
# return L.marker(latlng, {icon: cross});
# }""")


draw_church = assign("""function(feature, latlng){
const icon_church = L.icon(
{iconUrl: `https://flagcdn.com/64x48/${feature.properties.iso2}.png`, 
iconSize: [64, 48]}
);
return L.marker(latlng, {icon: cross});
}""")



app.layout = html.Div(children=[
    dl.Map([
    dl.TileLayer(),
    dl.GeoJSON(data=churches,
               id="church", 
               cluster=True, 
               options=dict(pointToLayer=draw_church),
               zoomToBoundsOnClick=True)
        ],
        center=[42.03, -93.64], zoom=12,
        style={'width': '100%', 'height': '50vh', 'margin': "auto", "display": "block"}
    )
])


    

app.run_server(mode='inline')

In [ ]:
import dash_html_components as html
import dash_leaflet as dl
import dash_core_components as dcc
import dash_leaflet.express as dlx
from dash import Dash
from dash.dependencies import Output, Input
from dash_extensions.javascript import assign

# A few cities in Denmark.
cities = [dict(name="Aalborg", lat=57.0268172, lon=9.837735),
          dict(name="Aarhus", lat=56.1780842, lon=10.1119354),
          dict(name="Copenhagen", lat=55.6712474, lon=12.5237848)]
# Create drop down options.
dd_options = [dict(value=c["name"], label=c["name"]) for c in cities]
dd_defaults = [o["value"] for o in dd_options]
# Generate geojson with a maker for each city and name as tooltip.
geojson = dlx.dicts_to_geojson([{**c, **dict(tooltip=c['name'])} for c in cities])
# Create javascript function that filters on feature name.
geojson_filter = assign("function(feature, context){return context.props.hideout.includes(feature.properties.name);}")
# Create example app.
app = JupyterDash(__name__)
app.layout = html.Div([
    dl.Map(children=[
        dl.TileLayer(),
        dl.GeoJSON(data=geojson, options=dict(filter=geojson_filter), hideout=dd_defaults, id="geojson")
    ], style={'width': '100%', 'height': '50vh', 'margin': "auto", "display": "block"}, id="map"),
    dcc.Dropdown(id="dd", value=dd_defaults, options=dd_options, clearable=False, multi=True)
])
# Link drop down to geojson hideout prop (could also be done with a normal callback).
app.clientside_callback("function(x){return x;}", Output("geojson", "hideout"), Input("dd", "value"))

app.run_server(mode='inline')
# if __name__ == '__main__':
#     app.run_server()